Las imágenes de alto rango dinámico (HDRI o HDR) son una técnica utilizada en imágenes y fotografía para reproducir un mayor rango dinámico de luminosidad que el que es posible con técnicas fotográficas o de imágenes digitales estándar. Si bien el ojo humano puede adaptarse a una amplia gama de condiciones de iluminación, la mayoría de los dispositivos de imágenes utilizan 8 bits por canal, por lo que estamos limitados a sólo 256 niveles. Cuando tomamos fotografías de una escena del mundo real, las regiones brillantes pueden estar sobreexpuestas, mientras que las oscuras pueden estar subexpuestas, por lo que no podemos capturar todos los detalles con una sola exposición. Las imágenes HDR funcionan con imágenes que utilizan más de 8 bits por canal (normalmente valores flotantes de 32 bits), lo que permite un rango dinámico mucho más amplio.

vVamos a ver la siguiente escena, donde tenemos 4 imágenes de exposición, con tiempos de exposición de: 15, 2,5, 1/4 y 1/30 segundos.

Cargar imágenes de exposición en una lista
La primera etapa consiste simplemente en cargar todas las imágenes en una lista. Además, necesitaremos los tiempos de exposición para los algoritmos HDR habituales. Preste atención a los tipos de datos, ya que las imágenes deben ser de 1 canal o 3 canales de 8 bits (np.uint8) y los tiempos de exposición deben ser flotantes32 y en segundos.

In [ ]:
import cv2 as cv
import numpy as np
# Loading exposure images into a list
img_fn = ["img0.jpg", "img1.jpg", "img2.jpg", "img3.jpg"]
img_list = [cv.imread(fn) for fn in img_fn]
exposure_times = np.array([15.0, 2.5, 0.25, 0.0333], dtype=np.float32)

Fusionar exposiciones en una imagen HDR
En esta etapa fusionamos la secuencia de exposición en una imagen HDR, mostrando 2 posibilidades que tenemos en OpenCV. El primer método es Debevec y el segundo es Robertson. Observe que la imagen HDR es de tipo float32 y no uint8, ya que contiene el rango dinámico completo de todas las imágenes de exposición.

In [ ]:
# Merge exposures to HDR image
merge_debevec = cv.createMergeDebevec()
hdr_debevec = merge_debevec.process(img_list, times=exposure_times.copy())
merge_robertson = cv.createMergeRobertson()
hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy())

Imagen HDR de mapa de tonos
Mapeamos los datos HDR flotantes de 32 bits en el rango [0..1]. De hecho, en algunos casos los valores pueden ser mayores que 1 o menores que 0, así que tenga en cuenta que luego tendremos que recortar los datos para evitar el desbordamiento.

In [ ]:
# Tonemap HDR image
tonemap1 = cv.createTonemap(gamma=2.2)
res_debevec = tonemap1.process(hdr_debevec.copy())

Fusionar exposiciones utilizando la fusión Mertens
Aquí mostramos un algoritmo alternativo para fusionar las imágenes de exposición, donde no necesitamos los tiempos de exposición. Tampoco necesitamos utilizar ningún algoritmo de mapa de tonos porque el algoritmo de Mertens ya nos da el resultado en el rango de [0..1]

In [ ]:
# Exposure fusion using Mertens
merge_mertens = cv.createMergeMertens()
res_mertens = merge_mertens.process(img_list)


Convierta a 8 bits y guarde
Para guardar o mostrar los resultados, necesitamos convertir los datos a enteros de 8 bits en el rango de [0..255].

In [ ]:
# Convert datatype to 8-bit and save
res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')
res_robertson_8bit = np.clip(res_robertson*255, 0, 255).astype('uint8')
res_mertens_8bit = np.clip(res_mertens*255, 0, 255).astype('uint8')
cv.imwrite("ldr_debevec.jpg", res_debevec_8bit)
cv.imwrite("ldr_robertson.jpg", res_robertson_8bit)
cv.imwrite("fusion_mertens.jpg", res_mertens_8bit)